In [ ]:
import random
import copy
import math

class GameEconomyBalancer:
    def __init__(self, shared_enemy_stats=True):
        """
        shared_enemy_stats: 
            True = Enemy health/damage/score same across all levels (RECOMMENDED)
            False = Each level has independent enemy parameters
        """
        self.shared_enemy_stats = shared_enemy_stats
        
        # Level configurations from your Lua files
        self.level_configs = {
            'level_01': {
                'duration': 60,
                'enemies': ['enemy1', 'enemy2'],
                'powerups': ['extra_life'],
                'player_health': 5,
            },
            'level_02': {
                'duration': 90,
                'enemies': ['enemy1', 'enemy2', 'enemy3'],
                'powerups': ['extra_life', 'nuke'],
                'player_health': 5,
            },
            'level_03': {
                'duration': 120,
                'enemies': ['enemy1', 'enemy2', 'enemy3'],
                'powerups': ['extra_life', 'nuke'],
                'player_health': 10,
                'has_boss': True,
            }
        }
        
        if shared_enemy_stats:
            # SHARED enemy parameters (consistent across all levels)
            self.shared_params = {
                # Enemy stats - same identity everywhere
                'enemy1_health': 3,
                'enemy1_score': 100,
                'enemy1_damage': 1,
                'enemy1_shoot_interval': 1.5,
                
                'enemy2_health': 2,
                'enemy2_score': 50,
                'enemy2_damage': 2,
                
                'enemy3_health': 6,
                'enemy3_score': 250,
                'enemy3_damage': 1,
                'enemy3_shoot_interval': 4.0,
                
                # Player constants
                'player_shoot_interval': 0.5,
                'player_damage': 1,
                
                # Boss stats (only in level 3)
                'boss_health': 40,
                'boss_score': 500,
            }
            
            # LEVEL-SPECIFIC parameters (difficulty scaling)
            self.level_params = {
                'level_01': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy2_spawn_rate': 5.0,
                    'extra_life_spawn_rate': 15.0,
                },
                'level_02': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy2_spawn_rate': 5.0,
                    'enemy3_spawn_rate': 12.0,
                    'extra_life_spawn_rate': 15.0,
                    'nuke_spawn_rate': 45.0,
                },
                'level_03': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy2_spawn_rate': 5.0,
                    'enemy3_spawn_rate': 12.0,
                    'extra_life_spawn_rate': 15.0,
                    'nuke_spawn_rate': 45.0,
                }
            }
            
            # Parameter ranges - SHARED (enemy stats)
            self.shared_ranges = {
                'enemy1_health': (1, 4),
                'enemy1_score': (50, 200),
                'enemy1_shoot_interval': (0.8, 3.0),
                'enemy1_damage': (1, 3),
                
                'enemy2_health': (1, 4),
                'enemy2_score': (25, 100),
                'enemy2_damage': (1, 4),
                
                'enemy3_health': (2, 6),
                'enemy3_score': (150, 400),
                'enemy3_shoot_interval': (2.0, 6.0),
                'enemy3_damage': (1, 1),
                
                'player_shoot_interval': (0.3, 1.0),
                'player_damage': (1, 1),
                
                'boss_health': (25, 60),
                'boss_score': (300, 1000),
            }
            
            # Parameter ranges - LEVEL SPECIFIC (spawn rates)
            self.level_ranges = {
                'enemy1_spawn_rate': (2.0, 10.0),
                'enemy2_spawn_rate': (2.0, 10.0),
                'enemy3_spawn_rate': (8.0, 20.0),
                'extra_life_spawn_rate': (10.0, 30.0),
                'nuke_spawn_rate': (30.0, 60.0),
            }
            
        else:
            # FULLY INDEPENDENT parameters per level
            self.level_params = {
                'level_01': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy1_health': 3,
                    'enemy1_score': 100,
                    'enemy1_shoot_interval': 1.5,
                    'enemy1_damage': 1,
                    
                    'enemy2_spawn_rate': 5.0,
                    'enemy2_health': 2,
                    'enemy2_score': 50,
                    'enemy2_damage': 2,
                    
                    'extra_life_spawn_rate': 15.0,
                    
                    'player_shoot_interval': 0.5,
                    'player_damage': 1,
                },
                'level_02': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy1_health': 3,
                    'enemy1_score': 100,
                    'enemy1_shoot_interval': 1.5,
                    'enemy1_damage': 1,
                    
                    'enemy2_spawn_rate': 5.0,
                    'enemy2_health': 2,
                    'enemy2_score': 50,
                    'enemy2_damage': 2,
                    
                    'enemy3_spawn_rate': 12.0,
                    'enemy3_health': 6,
                    'enemy3_score': 250,
                    'enemy3_shoot_interval': 4.0,
                    'enemy3_damage': 1,
                    
                    'extra_life_spawn_rate': 15.0,
                    'nuke_spawn_rate': 45.0,
                    
                    'player_shoot_interval': 0.5,
                    'player_damage': 1,
                },
                'level_03': {
                    'enemy1_spawn_rate': 5.0,
                    'enemy1_health': 3,
                    'enemy1_score': 100,
                    'enemy1_shoot_interval': 1.5,
                    #'enemy1_damage': 1,
                    
                    'enemy2_spawn_rate': 5.0,
                    'enemy2_health': 2,
                    'enemy2_score': 50,
                    #'enemy2_damage': 2,
                    
                    'enemy3_spawn_rate': 12.0,
                    'enemy3_health': 6,
                    'enemy3_score': 250,
                    'enemy3_shoot_interval': 4.0,
                    #'enemy3_damage': 1,
                    
                    'extra_life_spawn_rate': 15.0,
                    'nuke_spawn_rate': 45.0,
                    
                    'player_shoot_interval': 0.5,
                    'player_damage': 1,
                    
                    'boss_health': 40,
                    'boss_score': 500,
                }
            }
            
            self.level_ranges = {
                'enemy1_spawn_rate': (2.0, 10.0),
                'enemy1_health': (1, 5),
                'enemy1_score': (50, 200),
                'enemy1_shoot_interval': (0.8, 3.0),
                'enemy1_damage': (1, 3),
                
                'enemy2_spawn_rate': (2.0, 10.0),
                'enemy2_health': (1, 4),
                'enemy2_score': (25, 100),
                'enemy2_damage': (1, 4),
                
                'enemy3_spawn_rate': (8.0, 20.0),
                'enemy3_health': (3, 10),
                'enemy3_score': (150, 400),
                'enemy3_shoot_interval': (2.0, 6.0),
                'enemy3_damage': (1, 1),
                
                
                'extra_life_spawn_rate': (10.0, 35.0),
                'nuke_spawn_rate': (35.0, 60.0),
                
                'player_shoot_interval': (0.3, 1.0),
                'player_damage': (1, 1),
                
                'boss_health': (25, 70),
                'boss_score': (300, 1000),
            }
        
        # Target objectives per level
        self.level_targets = {
            'level_01': {
                'survival_time': 60,
                'avg_enemies': 3,
                'difficulty': 0.12,
                'score_rate': 80
            },
            'level_02': {
                'survival_time': 90,
                'avg_enemies': 5,
                'difficulty': 0.18,
                'score_rate': 120
            },
            'level_03': {
                'survival_time': 120,
                'avg_enemies': 4,
                'difficulty': 0.20,
                'score_rate': 150
            }
        }
        
        # EA parameters
        self.population_size = 60
        self.max_generations = 300
        self.mutation_rate = 0.1
        self.crossover_rate = 0.7
    
    def get_full_params(self, level_name, level_specific_params, shared_params=None):
        """Combine shared and level-specific parameters"""
        if self.shared_enemy_stats and shared_params:
            full_params = copy.deepcopy(shared_params)
            full_params.update(level_specific_params)
            # Add player health from level config
            full_params['player_health'] = self.level_configs[level_name]['player_health']
            return full_params
        else:
            full_params = copy.deepcopy(level_specific_params)
            full_params['player_health'] = self.level_configs[level_name]['player_health']
            return full_params
    
    def calculate_hit_chance(self, target_width, target_speed, bullet_speed, distance, screen_width, is_aimed=True):
        travel_time = distance / bullet_speed
        hitbox_factor = (target_width * 2) / screen_width
        relative_velocity = target_speed / bullet_speed
        aim_decay = 1.0 / (1.0 + relative_velocity * travel_time * 2.0)
        max_displacement = target_speed * travel_time
        evasion_factor = min(0.85, max_displacement / screen_width)
        aim_modifier = 1.0 if is_aimed else 0.6
        hit_chance = hitbox_factor * aim_decay * (1.0 - evasion_factor) * aim_modifier
        return max(0.02, min(0.95, hit_chance))
    
    def simulate_level(self, level_name, params):
        """Simulate a single level based on GEEvo framework"""
        config = self.level_configs[level_name]
        duration = config['duration']
        dt = 1.0
        
        SCREEN_WIDTH = 1200
        SCREEN_HEIGHT = 800
        PLAYER_BULLET_SPEED = 400
        ENEMY_BULLET_SPEED = 400
        
        player_health = params['player_health']
        score = 0
        enemies_killed = 0
        damage_received = 0
        
        # Spawn timers
        timers = {
            'enemy1': 0,
            'enemy2': 0,
            'enemy3': 0 if 'enemy3' in config['enemies'] else None,
            'extra_life': 0,
            'nuke': 0 if 'nuke' in config['powerups'] else None
        }
        
        active_enemies = {'enemy1': 0, 'enemy2': 0, 'enemy3': 0}
        
        boss_spawned = False
        boss_active = False
        if config.get('has_boss'):
            boss_health = params['boss_health']
        
        simulation_data = []
        
        for t in range(duration):
            # Enemy spawning
            for enemy_type in config['enemies']:
                timer_key = enemy_type
                timers[timer_key] += dt
                spawn_rate = params[f'{enemy_type}_spawn_rate']
                
                if timers[timer_key] >= spawn_rate:
                    if enemy_type == 'enemy3' and active_enemies['enemy3'] < 2:
                        active_enemies['enemy3'] += 1
                    elif enemy_type != 'enemy3':
                        active_enemies[enemy_type] += 1
                    timers[timer_key] = 0
            
            # Power-up spawning
            if 'extra_life' in config['powerups']:
                timers['extra_life'] += dt
                if timers['extra_life'] >= params['extra_life_spawn_rate']:
                    if random.random() > 0.4:
                        player_health = min(player_health + 1, params['player_health'] + 3)
                    timers['extra_life'] = 0
            
            if 'nuke' in config['powerups']:
                timers['nuke'] += dt
                if timers['nuke'] >= params['nuke_spawn_rate']:
                    if random.random() > 0.2:
                        for enemy_type in active_enemies:
                            enemies_killed += active_enemies[enemy_type]
                            score += active_enemies[enemy_type] * params.get(f'{enemy_type}_score', 0)
                        active_enemies = {'enemy1': 0, 'enemy2': 0, 'enemy3': 0}
                    timers['nuke'] = 0
            
            # Boss spawning (Level 3)
            if config.get('has_boss') and t == 30 and not boss_spawned:
                boss_spawned = True
                boss_active = True
                boss_health = params['boss_health']
            
            # Combat simulation
            player_shots_per_sec = 1.0 / params['player_shoot_interval']
            avg_distance = SCREEN_HEIGHT / 2
            
            # Player kills enemies
            for enemy_type in ['enemy1', 'enemy2', 'enemy3']:
                if active_enemies[enemy_type] > 0 and enemy_type in config['enemies']:
                    if enemy_type == 'enemy1':
                        hit_chance = self.calculate_hit_chance(64, 150, PLAYER_BULLET_SPEED, avg_distance, SCREEN_WIDTH)
                    elif enemy_type == 'enemy2':
                        hit_chance = self.calculate_hit_chance(64, 100, PLAYER_BULLET_SPEED, avg_distance, SCREEN_WIDTH)
                    else:  # enemy3
                        hit_chance = self.calculate_hit_chance(96, 100, PLAYER_BULLET_SPEED, avg_distance, SCREEN_WIDTH)
                    
                    if random.random() < player_shots_per_sec * hit_chance * dt:
                        active_enemies[enemy_type] = max(0, active_enemies[enemy_type] - 1)
                        enemies_killed += 1
                        score += params.get(f'{enemy_type}_score', 0)
            
            # Boss combat
            if config.get('has_boss') and boss_active:
                boss_bullets_per_sec = (16/3.0) + (1/1.5) + (12/2.5)
                boss_hit_chance = self.calculate_hit_chance(32, 400, 300, avg_distance, SCREEN_WIDTH)
                boss_damage_chance = boss_bullets_per_sec * boss_hit_chance * dt
                
                if t % 20 == 0:
                    active_enemies['enemy1'] += 3
                    active_enemies['enemy2'] += 3
                    active_enemies['enemy3'] += 3
                
                if random.random() < boss_damage_chance and player_health > 0:
                    player_health -= 1
                    damage_received += 1
                
                boss_as_target_hit_chance = self.calculate_hit_chance(100, 0, PLAYER_BULLET_SPEED, avg_distance * 0.6, SCREEN_WIDTH)
                if random.random() < player_shots_per_sec * boss_as_target_hit_chance * dt:
                    boss_health -= 1
                    if boss_health <= 0:
                        boss_active = False
                        score += params['boss_score']
            
            # Enemies damage player
            total_damage_chance = 0
            
            if 'enemy1' in config['enemies']:
                enemy1_bullets_per_sec = active_enemies['enemy1'] / params['enemy1_shoot_interval']
                enemy_to_player_hit_chance = self.calculate_hit_chance(32, 400, ENEMY_BULLET_SPEED, avg_distance, SCREEN_WIDTH)
                total_damage_chance += enemy1_bullets_per_sec * enemy_to_player_hit_chance * dt
            
            if 'enemy3' in config['enemies']:
                enemy3_bullets_per_sec = (active_enemies['enemy3'] * 8) / params['enemy3_shoot_interval']
                enemy3_spread_hit_chance = self.calculate_hit_chance(32, 400, ENEMY_BULLET_SPEED, avg_distance, SCREEN_WIDTH, False)
                total_damage_chance += enemy3_bullets_per_sec * enemy3_spread_hit_chance * dt
            
            if 'enemy2' in config['enemies']:
                enemy2_contact_chance = active_enemies['enemy2'] * 0.05 * dt
                total_damage_chance += enemy2_contact_chance
            
            if random.random() < total_damage_chance and player_health > 0:
                player_health -= 1
                damage_received += 1
            
            # Record state
            total_active = sum(active_enemies.values())
            simulation_data.append({
                'time': t,
                'player_health': max(0, player_health),
                'score': score,
                'active_enemies': total_active,
                'enemies_killed': enemies_killed,
                'damage_received': damage_received,
                'boss_active': boss_active if config.get('has_boss') else False
            })
            
            if player_health <= 0:
                break
        
        return simulation_data
    
    def calculate_fitness_for_level(self, level_name, simulation_data):
        """Calculate fitness for a single level"""
        if not simulation_data:
            return 0.0
        
        targets = self.level_targets[level_name]
        survival_time = len(simulation_data)
        final_data = simulation_data[-1]
        
        avg_enemies = sum(d['active_enemies'] for d in simulation_data) / len(simulation_data)
        difficulty = final_data['damage_received'] / max(1, survival_time)
        score_rate = final_data['score'] / max(1, survival_time)
        
        def prop(actual, target):
            if target == 0:
                return 0
            if actual > target:
                return target / max(actual, 0.001)
            else:
                return actual / max(target, 0.001)
        
        survival_fitness = prop(survival_time, targets['survival_time'])
        enemy_fitness = prop(avg_enemies, targets['avg_enemies'])
        difficulty_fitness = prop(difficulty, targets['difficulty'])
        score_fitness = prop(score_rate, targets['score_rate'])
        
        alpha = 0.05
        fitness = alpha + 0.3 * survival_fitness + 0.2 * enemy_fitness + \
                  0.3 * difficulty_fitness + 0.2 * score_fitness
        
        return fitness
    
    def calculate_overall_fitness(self, individual):
        """Calculate combined fitness across all three levels"""
        total_fitness = 0
        level_fitnesses = {}
        
        if self.shared_enemy_stats:
            shared_params = individual['shared']
            level_params = individual['levels']
        else:
            level_params = individual
        
        for level_name in ['level_01', 'level_02', 'level_03']:
            if self.shared_enemy_stats:
                params = self.get_full_params(level_name, level_params[level_name], shared_params)
            else:
                params = self.get_full_params(level_name, level_params[level_name])
            
            sim_data = self.simulate_level(level_name, params)
            fitness = self.calculate_fitness_for_level(level_name, sim_data)
            level_fitnesses[level_name] = fitness
            total_fitness += fitness
        
        overall_fitness = total_fitness / 3.0
        return overall_fitness, level_fitnesses
    
    def clamp_parameter(self, key, value, ranges_dict):
        if key in ranges_dict:
            min_val, max_val = ranges_dict[key]
            return max(min_val, min(max_val, value))
        return value
    
    def validate_parameters(self, params, ranges_dict):
        validated = {}
        for key, value in params.items():
            validated[key] = self.clamp_parameter(key, value, ranges_dict)
        return validated
    
    def mutate(self, individual):
        mutated = copy.deepcopy(individual)
        
        if self.shared_enemy_stats:
            # Mutate shared parameters
            if random.random() < self.mutation_rate:
                mutable_keys = [k for k in mutated['shared'].keys() if k in self.shared_ranges]
                if mutable_keys:
                    key = random.choice(mutable_keys)
                    change = random.uniform(-0.3, 0.3)
                    mutated['shared'][key] = mutated['shared'][key] + mutated['shared'][key] * change
            
            mutated['shared'] = self.validate_parameters(mutated['shared'], self.shared_ranges)
            
            # Mutate level-specific parameters
            for level_name in mutated['levels']:
                if random.random() < self.mutation_rate:
                    mutable_keys = [k for k in mutated['levels'][level_name].keys() if k in self.level_ranges]
                    if mutable_keys:
                        key = random.choice(mutable_keys)
                        change = random.uniform(-0.3, 0.3)
                        mutated['levels'][level_name][key] = mutated['levels'][level_name][key] + mutated['levels'][level_name][key] * change
                
                mutated['levels'][level_name] = self.validate_parameters(mutated['levels'][level_name], self.level_ranges)
        else:
            # Mutate all parameters independently per level
            for level_name in mutated:
                if random.random() < self.mutation_rate:
                    mutable_keys = [k for k in mutated[level_name].keys() if k in self.level_ranges]
                    if mutable_keys:
                        key = random.choice(mutable_keys)
                        change = random.uniform(-0.3, 0.3)
                        mutated[level_name][key] = mutated[level_name][key] + mutated[level_name][key] * change
                
                mutated[level_name] = self.validate_parameters(mutated[level_name], self.level_ranges)
        
        return mutated
    
    def crossover(self, parent1, parent2):
        child = {}
        
        if self.shared_enemy_stats:
            # Crossover shared parameters
            child['shared'] = {}
            for key in parent1['shared'].keys():
                if random.random() < self.crossover_rate:
                    operation = random.choice(['choose_p1', 'choose_p2', 'add', 'subtract'])
                    
                    if operation == 'choose_p1':
                        child['shared'][key] = parent1['shared'][key]
                    elif operation == 'choose_p2':
                        child['shared'][key] = parent2['shared'][key]
                    elif operation == 'add':
                        child['shared'][key] = parent1['shared'][key] + parent2['shared'][key]
                    else:
                        child['shared'][key] = abs(parent1['shared'][key] - parent2['shared'][key])
                else:
                    child['shared'][key] = parent1['shared'][key]
            
            child['shared'] = self.validate_parameters(child['shared'], self.shared_ranges)
            
            # Crossover level-specific parameters
            child['levels'] = {}
            for level_name in parent1['levels']:
                child['levels'][level_name] = {}
                for key in parent1['levels'][level_name].keys():
                    if random.random() < self.crossover_rate:
                        operation = random.choice(['choose_p1', 'choose_p2', 'add', 'subtract'])
                        
                        if operation == 'choose_p1':
                            child['levels'][level_name][key] = parent1['levels'][level_name][key]
                        elif operation == 'choose_p2':
                            child['levels'][level_name][key] = parent2['levels'][level_name][key]
                        elif operation == 'add':
                            child['levels'][level_name][key] = parent1['levels'][level_name][key] + parent2['levels'][level_name][key]
                        else:
                            child['levels'][level_name][key] = abs(parent1['levels'][level_name][key] - parent2['levels'][level_name][key])
                    else:
                        child['levels'][level_name][key] = parent1['levels'][level_name][key]
                
                child['levels'][level_name] = self.validate_parameters(child['levels'][level_name], self.level_ranges)
        else:
            # Crossover all parameters independently
            for level_name in parent1:
                child[level_name] = {}
                for key in parent1[level_name].keys():
                    if random.random() < self.crossover_rate:
                        operation = random.choice(['choose_p1', 'choose_p2', 'add', 'subtract'])
                        
                        if operation == 'choose_p1':
                            child[level_name][key] = parent1[level_name][key]
                        elif operation == 'choose_p2':
                            child[level_name][key] = parent2[level_name][key]
                        elif operation == 'add':
                            child[level_name][key] = parent1[level_name][key] + parent2[level_name][key]
                        else:
                            child[level_name][key] = abs(parent1[level_name][key] - parent2[level_name][key])
                    else:
                        child[level_name][key] = parent1[level_name][key]
                
                child[level_name] = self.validate_parameters(child[level_name], self.level_ranges)
        
        return child
    
    def evolutionary_balance(self):
        mode = "HYBRID (Shared Enemy Stats)" if self.shared_enemy_stats else "INDEPENDENT (Per-Level Stats)"
        print("\n" + "="*80)
        print(f"GEEvo: Multi-Level Game Economy Balancer - {mode}")
        print("="*80)
        
        # Initialize population
        population = []
        for _ in range(self.population_size):
            if self.shared_enemy_stats:
                individual = {
                    'shared': copy.deepcopy(self.shared_params),
                    'levels': {}
                }
                
                # Add variation to shared params
                for key in individual['shared'].keys():
                    if key in self.shared_ranges:
                        min_val, max_val = self.shared_ranges[key]
                        variation = random.uniform(0.8, 1.2)
                        individual['shared'][key] = self.clamp_parameter(
                            key, individual['shared'][key] * variation, self.shared_ranges
                        )
                
                # Initialize level-specific params
                for level_name in self.level_params:
                    individual['levels'][level_name] = copy.deepcopy(self.level_params[level_name])
                    for key in individual['levels'][level_name].keys():
                        if key in self.level_ranges:
                            min_val, max_val = self.level_ranges[key]
                            variation = random.uniform(0.8, 1.2)
                            individual['levels'][level_name][key] = self.clamp_parameter(
                                key, individual['levels'][level_name][key] * variation, self.level_ranges
                            )
            else:
                individual = {}
                for level_name in self.level_params:
                    individual[level_name] = copy.deepcopy(self.level_params[level_name])
                    for key in individual[level_name].keys():
                        if key in self.level_ranges:
                            min_val, max_val = self.level_ranges[key]
                            variation = random.uniform(0.8, 1.2)
                            individual[level_name][key] = self.clamp_parameter(
                                key, individual[level_name][key] * variation, self.level_ranges
                            )
            
            population.append(individual)
        
        best_fitness = 0
        best_individual = None
        best_level_fitnesses = None
        
        for generation in range(self.max_generations):
            fitness_scores = []
            for individual in population:
                overall_fitness, level_fitnesses = self.calculate_overall_fitness(individual)
                fitness_scores.append((overall_fitness, individual, level_fitnesses))
            
            fitness_scores.sort(reverse=True, key=lambda x: x[0])
            
            if fitness_scores[0][0] > best_fitness:
                best_fitness = fitness_scores[0][0]
                best_individual = copy.deepcopy(fitness_scores[0][1])
                best_level_fitnesses = fitness_scores[0][2]
            
            print(f"Gen {generation + 1}/{self.max_generations}: "
                  f"Overall Fitness = {best_fitness:.4f} | "
                  f"L1: {best_level_fitnesses['level_01']:.3f}, "
                  f"L2: {best_level_fitnesses['level_02']:.3f}, "
                  f"L3: {best_level_fitnesses['level_03']:.3f}")
            
            if best_fitness >= 1.05:
                break
            
            survivors = [ind for _, ind, _ in fitness_scores[:self.population_size // 2]]
            new_population = survivors.copy()
            
            while len(new_population) < self.population_size:
                if random.random() < self.crossover_rate:
                    parent1 = random.choice(survivors)
                    parent2 = random.choice(survivors)
                    child = self.crossover(parent1, parent2)
                else:
                    child = copy.deepcopy(random.choice(survivors))
                
                child = self.mutate(child)
                new_population.append(child)
            
            population = new_population
        
        return best_individual, best_fitness, best_level_fitnesses
    
    def display_results(self, optimal_individual, fitness, level_fitnesses):
        print("\n" + "="*80)
        print("OPTIMAL PARAMETERS - ALL LEVELS")
        print("="*80)
        print(f"Overall Fitness Score: {fitness:.4f} (Target: 1.05+)")
        print(f"Level 01 Fitness: {level_fitnesses['level_01']:.4f}")
        print(f"Level 02 Fitness: {level_fitnesses['level_02']:.4f}")
        print(f"Level 03 Fitness: {level_fitnesses['level_03']:.4f}")
        print()
        
        if self.shared_enemy_stats:
            print("\n" + "="*80)
            print("SHARED ENEMY PARAMETERS (Consistent Across All Levels)")
            print("="*80)
            shared = optimal_individual['shared']
            
            print(f"\nEnemy 1 (Horizontal Shooter):")
            print(f"  Health: {shared['enemy1_health']:.0f}")
            print(f"  Score: {shared['enemy1_score']:.0f}")
            print(f"  Damage: {shared['enemy1_damage']:.0f}")
            print(f"  Shoot Interval: {shared['enemy1_shoot_interval']:.2f}s")
            
            print(f"\nEnemy 2 (Contact Chaser):")
            print(f"  Health: {shared['enemy2_health']:.0f}")
            print(f"  Score: {shared['enemy2_score']:.0f}")
            print(f"  Damage: {shared['enemy2_damage']:.0f}")
            
            print(f"\nEnemy 3 (8-Way Shooter):")
            print(f"  Health: {shared['enemy3_health']:.0f}")
            print(f"  Score: {shared['enemy3_score']:.0f}")
            print(f"  Damage: {shared['enemy3_damage']:.0f}")
            print(f"  Shoot Interval: {shared['enemy3_shoot_interval']:.2f}s")
            
            print(f"\nPlayer Stats:")
            print(f"  Shoot Interval: {shared['player_shoot_interval']:.2f}s")
            print(f"  Damage: {shared['player_damage']:.0f}")
            
            print(f"\nBoss Stats (Level 3):")
            print(f"  Health: {shared['boss_health']:.0f}")
            print(f"  Score: {shared['boss_score']:.0f}")
        
        for level_name in ['level_01', 'level_02', 'level_03']:
            config = self.level_configs[level_name]
            
            if self.shared_enemy_stats:
                level_params = optimal_individual['levels'][level_name]
                full_params = self.get_full_params(level_name, level_params, optimal_individual['shared'])
            else:
                level_params = optimal_individual[level_name]
                full_params = self.get_full_params(level_name, level_params)
            
            print(f"\n{'='*80}")
            print(f"{level_name.upper()} - Duration: {config['duration']}s, Player Health: {config['player_health']}")
            print(f"{'='*80}")
            
            if self.shared_enemy_stats:
                print("\nSPAWN RATES (Difficulty Scaling):")
                print(f"  Enemy 1 Spawn Rate: {level_params['enemy1_spawn_rate']:.2f}s")
                print(f"  Enemy 2 Spawn Rate: {level_params['enemy2_spawn_rate']:.2f}s")
                if 'enemy3' in config['enemies']:
                    print(f"  Enemy 3 Spawn Rate: {level_params['enemy3_spawn_rate']:.2f}s")
                
                print(f"\nPOWER-UPS:")
                print(f"  Extra Life Spawn Rate: {level_params['extra_life_spawn_rate']:.2f}s")
                if 'nuke' in config['powerups']:
                    print(f"  Nuke Spawn Rate: {level_params['nuke_spawn_rate']:.2f}s")
            else:
                print("\nENEMY PARAMETERS (Level-Specific):")
                print(f"\n  Enemy 1:")
                print(f"    Spawn Rate: {level_params['enemy1_spawn_rate']:.2f}s")
                print(f"    Health: {level_params['enemy1_health']:.0f}")
                print(f"    Score: {level_params['enemy1_score']:.0f}")
                print(f"    Shoot Interval: {level_params['enemy1_shoot_interval']:.2f}s")
                
                print(f"\n  Enemy 2:")
                print(f"    Spawn Rate: {level_params['enemy2_spawn_rate']:.2f}s")
                print(f"    Health: {level_params['enemy2_health']:.0f}")
                print(f"    Score: {level_params['enemy2_score']:.0f}")
                
                if 'enemy3' in config['enemies']:
                    print(f"\n  Enemy 3:")
                    print(f"    Spawn Rate: {level_params['enemy3_spawn_rate']:.2f}s")
                    print(f"    Health: {level_params['enemy3_health']:.0f}")
                    print(f"    Score: {level_params['enemy3_score']:.0f}")
                    print(f"    Shoot Interval: {level_params['enemy3_shoot_interval']:.2f}s")
                
                print(f"\n  Power-ups:")
                print(f"    Extra Life Spawn Rate: {level_params['extra_life_spawn_rate']:.2f}s")
                if 'nuke' in config['powerups']:
                    print(f"    Nuke Spawn Rate: {level_params['nuke_spawn_rate']:.2f}s")
                
                print(f"\n  Player:")
                print(f"    Shoot Interval: {level_params['player_shoot_interval']:.2f}s")
                
                if level_name == 'level_03':
                    print(f"\n  Boss:")
                    print(f"    Health: {level_params['boss_health']:.0f}")
                    print(f"    Score: {level_params['boss_score']:.0f}")
            
            # Test simulation
            print(f"\nSIMULATION TEST:")
            sim_data = self.simulate_level(level_name, full_params)
            if sim_data:
                final = sim_data[-1]
                avg_enemies = sum(d['active_enemies'] for d in sim_data) / len(sim_data)
                targets = self.level_targets[level_name]
                
                print(f"  Survival Time: {len(sim_data)}s / {targets['survival_time']}s target")
                print(f"  Final Score: {final['score']}")
                print(f"  Enemies Killed: {final['enemies_killed']}")
                print(f"  Avg Active Enemies: {avg_enemies:.2f} / {targets['avg_enemies']} target")
                print(f"  Damage Received: {final['damage_received']} ({final['damage_received']/len(sim_data):.2f}/s vs {targets['difficulty']:.2f} target)")
                print(f"  Score Rate: {final['score']/len(sim_data):.1f}/s vs {targets['score_rate']:.1f} target")
                if level_name == 'level_03':
                    boss_status = "Defeated" if not final['boss_active'] else "Still Active"
                    print(f"  Boss Status: {boss_status}")


def main():
    print("="*80)
    print("GALACTIC SIEGE - MULTI-LEVEL GAME ECONOMY BALANCER")
    print("Based on GEEvo: Game Economy Generation and Balancing")
    print("with Evolutionary Algorithms")
    print("="*80)
    
    while True:
        print("\nSelect Balancing Mode:")
        print("1. Execute - Shared enemy stats, level-specific spawn rates")
        print("2. Exit")
        
        choice = input("\nEnter choice: ").strip()
        
        if choice == '1':
            print("\n>>> EXECUTING: <<<")
            print("Enemy health/damage/score will be CONSISTENT across all levels.")
            print("Spawn rates and powerups will VARY to create difficulty progression.\n")
            balancer = GameEconomyBalancer(shared_enemy_stats=True)
            optimal_individual, fitness, level_fitnesses = balancer.evolutionary_balance()
            balancer.display_results(optimal_individual, fitness, level_fitnesses)
            
        elif choice == '2':
            print("\nExiting...")
            break
        else:
            print("Invalid choice. Please enter 1 or 2.")


if __name__ == "__main__":
    main()

#This version

GALACTIC SIEGE - MULTI-LEVEL GAME ECONOMY BALANCER
Based on GEEvo: Game Economy Generation and Balancing
with Evolutionary Algorithms

Select Balancing Mode:
1. Execute - Shared enemy stats, level-specific spawn rates
2. Exit



Enter choice:  1



>>> EXECUTING: <<<
Enemy health/damage/score will be CONSISTENT across all levels.
Spawn rates and powerups will VARY to create difficulty progression.


GEEvo: Multi-Level Game Economy Balancer - HYBRID (Shared Enemy Stats)
Gen 1/300: Overall Fitness = 0.7265 | L1: 0.732, L2: 0.675, L3: 0.772
Gen 2/300: Overall Fitness = 0.7476 | L1: 0.824, L2: 0.694, L3: 0.725
Gen 3/300: Overall Fitness = 0.7476 | L1: 0.824, L2: 0.694, L3: 0.725
Gen 4/300: Overall Fitness = 0.7476 | L1: 0.824, L2: 0.694, L3: 0.725
Gen 5/300: Overall Fitness = 0.7578 | L1: 0.790, L2: 0.736, L3: 0.747
Gen 6/300: Overall Fitness = 0.7578 | L1: 0.790, L2: 0.736, L3: 0.747
Gen 7/300: Overall Fitness = 0.7711 | L1: 0.786, L2: 0.768, L3: 0.760
Gen 8/300: Overall Fitness = 0.7711 | L1: 0.786, L2: 0.768, L3: 0.760
Gen 9/300: Overall Fitness = 0.7711 | L1: 0.786, L2: 0.768, L3: 0.760
Gen 10/300: Overall Fitness = 0.7711 | L1: 0.786, L2: 0.768, L3: 0.760
Gen 11/300: Overall Fitness = 0.7711 | L1: 0.786, L2: 0.768, L3: 0.760
Ge

In [ ]:
1